In [1]:
from neo4j import GraphDatabase
url = "bolt://localhost:7687"
username = "neo4j"
password= "neo4j"

from langchain_neo4j import Neo4jGraph
graph = Neo4jGraph(url=url, username=username, password=password)
driver = GraphDatabase.driver(url, auth=(username, password))
session = driver.session()

import ipywidgets as widgets
from IPython.display import display, clear_output
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase


In [ ]:
from yfiles_jupyter_graphs import GraphWidget
import ipywidgets as widgets
from IPython.display import display

def showGraph(query, layout):
    # Create an Output widget to capture and display the graph
    out = widgets.Output(layout={'border': '1px solid black'})
    
    # Create the GraphWidget inside the Output context
    with out:
        session = driver.session()
        graph_data = session.run(query).graph()
        widget = GraphWidget(graph=graph_data)
        widget.set_sidebar(start_with='Data')

        # Define the update function for resizing
        def update_graph_size(change, dimension):
            new_size = change['new']
            if dimension == 'width':
                widget.layout.width = f'{new_size}px'
            elif dimension == 'height':
                widget.layout.height = f'{new_size}px'

        # Create sliders for width and height
        width_slider = widgets.FloatSlider(
            value=2550,  # Initial width
            min=400,    # Minimum width
            max=2550,   # Maximum width
            step=50,    # Step size
            description='Width:',
            continuous_update=True
        )
        height_slider = widgets.FloatSlider(
            value=1200,  # Initial height
            min=400,    # Minimum height
            max=1200,   # Maximum height
            step=50,    # Step size
            description='Height:',
            continuous_update=True
        )

        # Link sliders to the update function
        width_slider.observe(lambda change: update_graph_size(change, 'width'), names='value')
        height_slider.observe(lambda change: update_graph_size(change, 'height'), names='value')
        widget.layout.width = '2550px'
        widget.layout.height = '1200px'
        
        # Apply the chosen layout
        if layout == "hierarchic":
            widget.hierarchic_layout()
        elif layout == "radial":
            widget.radial_layout()
            widget.node_label_mapping = 'uri'

        # Display the sliders and the graph
        display(width_slider, height_slider, widget)
    
    return out

# Create Tab widget to display the two graphs on separate pages
tab = widgets.Tab()

# Define the queries and layouts
graph1 = showGraph("""
    MATCH (a:Taxonomy)-[r]->(b)
    RETURN a, r, b
          """, "hierarchic")
graph2 = showGraph("""
    MATCH (a)
    WHERE a.uri in ["709942bd_alexnet"]
    MATCH (a)-[r*1..7]->(b)
    RETURN a, r, b
          """, "radial")
graph3 = showGraph("""
    MATCH (a)-[r]->(b)
    RETURN a, r, b
          """, "radial")

# Set the tabs to contain the two graphs
tab.children = [graph1, graph2, graph3]

# Set the tab titles
tab.set_title(0, 'Taxonomy')
tab.set_title(1, 'Ontology')
tab.set_title(2, 'Test')
# Display the tab widget
display(tab)

Used to embed onto streamlit, also starts voila

voila yfiles_graph.ipynb --Voila.tornado_settings="{'headers':{'Content-Security-Policy':'frame-ancestors *'}}"

<!-- MATCH (a)
WHERE a.uri in ["AAE","GAN"]
MATCH (a)-[r*1..2]->(b)
RETURN a, r, b -->

<!-- MATCH (a)-[r:ns0__hasTaskType]-(b)
RETURN a, r, b -->


<!-- MATCH (a)
    WHERE a.uri in ["AAE","GAN"]
    MATCH (a)-[r*1..2]->(b)
    RETURN a, r, b


MATCH (n)-[r]->(m) RETURN n, r, m -->

<!-- MATCH (a)-[r:ns0__hasTaskType]->(b)
RETURN a, r, b; -->